# CS2 Demo Analysis Notebook

This notebook demonstrates how to use the CS2 Demo Analyzer backend for detailed demo analysis.

## Overview

The CS2 Demo Analyzer provides comprehensive tools for parsing Counter-Strike 2 demo files and extracting meaningful player statistics. This notebook shows:

- How to parse demo files
- Extract and analyze player statistics
- Calculate custom player ratings
- Visualize performance data
- Export results for further analysis

## Setup

First, let's import the necessary modules and set up our environment.

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Add the backend directory to the path so we can import our modules
sys.path.append('../backend')

# Import our custom modules
from parser import DemoParser, parse_demo_file
from rating import PlayerRating, calculate_player_ratings, analyze_match_ratings

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Setup complete! Ready to analyze CS2 demos.")

## Demo File Parsing

Let's start by parsing a demo file. You'll need to have a `.dem` file in the `data/` directory.

In [ ]:
# List available demo files
demo_dir = Path('../data')
demo_files = list(demo_dir.glob('*.dem'))

if demo_files:
    print(f"Found {len(demo_files)} demo file(s):")
    for demo_file in demo_files:
        print(f"  - {demo_file.name}")
        
    # Use the first demo file for analysis
    demo_path = str(demo_files[0])
    print(f"\nUsing: {demo_path}")
else:
    print("No demo files found in ../data/ directory.")
    print("Please add a .dem file to continue.")
    demo_path = None

In [ ]:
# Parse the demo file
if demo_path:
    print(f"Parsing demo: {demo_path}")
    
    try:
        # Parse the demo
        result = parse_demo_file(demo_path)
        
        # Extract data
        demo_data = result['parsed_data']
        stats_data = result['stats_data']
        player_stats = stats_data['stats']
        
        print(f"✅ Successfully parsed demo!")
        print(f"   Map: {demo_data['header']['map_name']}")
        print(f"   Server: {demo_data['header']['server_name']}")
        print(f"   Players: {len(player_stats)}")
        print(f"   Events: {len(stats_data['events'])}")
        
    except Exception as e:
        print(f"❌ Error parsing demo: {e}")
        player_stats = None
else:
    player_stats = None

## Player Statistics Overview

Let's examine the extracted player statistics.

In [ ]:
# Display player statistics
if player_stats is not None:
    print("Player Statistics Overview:")
    display(player_stats)
    
    # Summary statistics
    print("\nSummary Statistics:")
    summary = player_stats.describe()
    display(summary)
else:
    print("No player statistics available.")

## Rating System Analysis

Now let's calculate player ratings using our custom rating system.

In [ ]:
# Calculate ratings with default weights
if player_stats is not None:
    print("Calculating player ratings...")
    
    # Default weights
    default_weights = {
        'kd_ratio': 0.25,
        'hs_percentage': 0.20,
        'adr': 0.20,
        'multi_kill_score': 0.15,
        'clutch_score': 0.20
    }
    
    # Calculate ratings
    rated_players = calculate_player_ratings(player_stats, default_weights)
    
    print("✅ Ratings calculated!")
    print("\nTop 5 players by rating:")
    top_5 = rated_players.nlargest(5, 'rating')[['name', 'rating', 'kd_ratio', 'hs_percentage', 'adr']]
    display(top_5)
else:
    rated_players = None

## Rating Breakdown Analysis

Let's examine how each metric contributes to the rating for the top player.

In [ ]:
# Analyze rating breakdown for top player
if rated_players is not None:
    top_player = rated_players.iloc[0]
    print(f"Rating breakdown for {top_player['name']}:")
    
    # Get detailed breakdown
    calculator = PlayerRating(default_weights)
    breakdown = calculator.get_rating_breakdown(top_player)
    
    # Display breakdown
    for metric, details in breakdown.items():
        if metric != 'final_rating':
            print(f"{metric}:")
            print(f"  Raw value: {details['raw_value']}")
            print(f"  Normalized score: {details['normalized_score']:.3f}")
            print(f"  Weight: {details['weight']}")
            print(f"  Contribution: {details['contribution']:.2f}")
            print()
    
    print(f"Final Rating: {breakdown['final_rating']}")
else:
    print("No rated players available.")

## Data Visualization

Let's create some visualizations to better understand the data.

In [ ]:
# Create visualizations
if rated_players is not None:
    # Set up the plotting area
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('CS2 Demo Analysis - Player Performance', fontsize=16, fontweight='bold')
    
    # 1. Rating Distribution
    axes[0, 0].hist(rated_players['rating'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Player Rating Distribution')
    axes[0, 0].set_xlabel('Rating')
    axes[0, 0].set_ylabel('Number of Players')
    axes[0, 0].axvline(rated_players['rating'].mean(), color='red', linestyle='--', label=f'Mean: {rated_players["rating"].mean():.1f}')
    axes[0, 0].legend()
    
    # 2. K/D vs Rating Scatter
    scatter = axes[0, 1].scatter(rated_players['kd_ratio'], rated_players['rating'], 
                                 alpha=0.6, s=50, c=rated_players['hs_percentage'], cmap='viridis')
    axes[0, 1].set_title('K/D Ratio vs Rating')
    axes[0, 1].set_xlabel('K/D Ratio')
    axes[0, 1].set_ylabel('Rating')
    plt.colorbar(scatter, ax=axes[0, 1], label='Headshot %')
    
    # 3. ADR vs Rating
    axes[1, 0].scatter(rated_players['adr'], rated_players['rating'], alpha=0.6, color='green', s=50)
    axes[1, 0].set_title('ADR vs Rating')
    axes[1, 0].set_xlabel('Average Damage per Round')
    axes[1, 0].set_ylabel('Rating')
    
    # 4. Top 10 Players Bar Chart
    top_10 = rated_players.nlargest(10, 'rating')
    bars = axes[1, 1].barh(range(len(top_10)), top_10['rating'])
    axes[1, 1].set_yticks(range(len(top_10)))
    axes[1, 1].set_yticklabels(top_10['name'])
    axes[1, 1].set_title('Top 10 Players by Rating')
    axes[1, 1].set_xlabel('Rating')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization.")

## Interactive Visualizations with Plotly

Let's create some interactive visualizations using Plotly.

In [ ]:
# Interactive scatter plot
if rated_players is not None:
    fig = px.scatter(rated_players, 
                     x='kd_ratio', 
                     y='rating',
                     size='adr',
                     color='hs_percentage',
                     hover_name='name',
                     title='Interactive Player Performance Analysis',
                     labels={
                         'kd_ratio': 'K/D Ratio',
                         'rating': 'Player Rating',
                         'adr': 'Average Damage per Round',
                         'hs_percentage': 'Headshot Percentage'
                     })
    
    fig.update_layout(
        width=800,
        height=600
    )
    
    fig.show()
else:
    print("No data available for interactive visualization.")

## Export Results

Finally, let's export our results for further analysis or sharing.

In [ ]:
# Export results
if rated_players is not None:
    # Create results directory if it doesn't exist
    results_dir = Path('../results')
    results_dir.mkdir(exist_ok=True)
    
    # Export to CSV
    csv_path = results_dir / 'notebook_analysis.csv'
    rated_players.to_csv(csv_path, index=True)
    print(f"✅ Exported player statistics to {csv_path}")
    
    # Export summary to JSON
    summary_data = {
        'match_info': {
            'map': demo_data['header']['map_name'],
            'server': demo_data['header']['server_name'],
            'total_players': len(rated_players)
        },
        'rating_summary': {
            'average_rating': rated_players['rating'].mean(),
            'median_rating': rated_players['rating'].median(),
            'top_rating': rated_players['rating'].max(),
            'rating_std': rated_players['rating'].std()
        },
        'top_performers': rated_players.nlargest(3, 'rating')[['name', 'rating']].to_dict('records')
    }
    
    import json
    json_path = results_dir / 'notebook_summary.json'
    with open(json_path, 'w') as f:
        json.dump(summary_data, f, indent=2)
    
    print(f"✅ Exported summary to {json_path}")
    print("\nAnalysis complete! Check the ../results/ directory for exported files.")
else:
    print("No data to export.")

## Conclusion

This notebook demonstrated the key features of the CS2 Demo Analyzer:

1. **Demo Parsing**: Using demoparser2 to extract comprehensive game data
2. **Statistics Calculation**: Computing advanced metrics like K/D, HS%, ADR, etc.
3. **Rating System**: Implementing a weighted rating algorithm with customizable parameters
4. **Data Visualization**: Creating both static and interactive charts for performance analysis
5. **Data Export**: Saving results in multiple formats for further use

### Next Steps

- Try analyzing different demo files
- Experiment with different rating weights
- Add more advanced visualizations (heatmaps, timelines, etc.)
- Compare multiple matches or tournaments

### Tips

- For best results, use demos from competitive matches
- The rating system can be fine-tuned based on your preferences
- Interactive visualizations work best in Jupyter Lab or VS Code

Happy analyzing! 🎯